<a href="https://colab.research.google.com/github/pbeens/Public-Health-Ontario-PDF-Scraper/blob/main/Public_Health_Ontario_PDF_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program scrapes the Ontario Public Health website and downloads all PDF files into the PDFs folder.

It has been written to be run in Colab only. 

TODO: The program has only been written to go two links deep, which may or may not be sufficient to download all PDFs.

TODO: Don't attempt to grab links from PDFs. It slows the program down too much!

TODO: Skips URLs with '?sc_lang=fr' (CHECK) (These are dupes)

The PDFs get zip'd and can be downloaded manually to your computer. 

URL of this notebook in Colab: https://colab.research.google.com/drive/1iC1l3ZHw6_vgztBUcSjkPsn5FVFlMdJ1?usp=sharing

In [ ]:
# imports
import time

In [ ]:
!mkdir 'PDFs'

In [ ]:
# setup takes a long time
# Note that this contains some custom lines to set
# the download folder

!pip install selenium -q
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

prefs = {'download.default_directory' : '/content/PDFs'}
chrome_options.add_experimental_option('prefs', prefs)

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
# global variable(s)
url = 'https://www.publichealthontario.ca/'
all_links = []

In [ ]:
# to filter out undesired links
exceptions = ['mailto', 'Login', '#']

def url_okay(url):
    for exception in exceptions:
        if url.count(exception) > 0:
            return False
    return True

In [ ]:
wd.get(url)

links = wd.find_elements_by_xpath("//a[@href]") # finds all links

for link in links:
    title = link.text
    url = link.get_attribute('href')
    if url_okay(url):
        if url.count('www.publichealthontario.ca') > 0:
            if url not in all_links:
                all_links.append(url)
                print(title, url)

In [ ]:
# grab links from all_links pages

for link in all_links:
    print(f'Grabbing links from {link}... ')
    wd.get(link)
    links_in_page = wd.find_elements_by_xpath("//a[@href]")
    for lnk in links_in_page: # added sorted() to this
        title = lnk.text
        url = lnk.get_attribute('href')
        if url_okay(url):
            if url.count('www.publichealthontario.ca') > 0:
                if url not in all_links:
                    all_links.append(url)
                    print(title, url)

In [ ]:
# download all PDFs
for link in sorted(all_links):
    if link.count('pdf') > 0:
        print(link.split('/')[-1]) # print filename only
        wd.get(link)
        time.sleep(1)

In [ ]:
# zip the files for download
import shutil
shutil.make_archive('PDFs', 'zip', '/content/PDFs')